In [10]:
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
#rag data 로드
rag_df=pd.read_csv('rag_dataset.csv',encoding='utf-8',index_col=0)

In [3]:
loader = DataFrameLoader(rag_df, page_content_column="rag_text")

In [6]:
# 벡터스토어에 임베딩할 모델 선언
model_name = 'intfloat/multilingual-e5-large'
model_kwargs = {"device":"cuda"}
encode_kwargs={'normalize_embeddings':True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs

)

2024-04-01 15:37:10.433544: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-01 15:37:10.451216: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 15:37:10.451234: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 15:37:10.451246: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-01 15:37:10.455112: I tensorflow/core/platform/cpu_feature_g

In [11]:
pages=loader.load_and_split()
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

# text_splitter로 chunksize, chunkoverlap 등을 설정해주는 텍스트 전처리를 함
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=150, length_function=tiktoken_len)
texts = text_splitter.split_documents(pages)
len(texts)

168900

In [ ]:
# 임베딩 모델 선언 후 Chroma db에 저장함
vector_db = Chroma.from_documents(texts, hf,persist_directory='./k-paper_db')
#Chroma에 저장하는 코드